<h1 align=center><font size = 6>IBM Applied Data Science Capstone Course by Coursera</font></h1>

<h1 align=center><font size = 5>RESTAURANT IN TORONTO</font></h1>

In [1]:
import numpy as np
import pandas as pd
import json 
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge bs4 --yes
#!conda install -c conda-forge folium=0.5.0 --yes
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [3]:
soup = BeautifulSoup(source, 'html.parser')

In [4]:
table=soup.find('table')

In [5]:
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [6]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [7]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h4 align=left><font size = 4>Drop cells with a borough that is Not assigned</font></h1>

In [8]:
toronto_df=df[df.Borough != "Not assigned"].reset_index(drop=True)

In [9]:
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
toronto_df_grouped = toronto_df.groupby(["Postalcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h4 align=left><font size = 4>cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.</font></h4>

In [11]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
toronto_df_grouped.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h4 align=left><font size = 4>Check whether it is the same as required by the question</font></h4>

In [12]:
column_names = ["Postalcode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_list:
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["Postalcode"]==postcode], ignore_index=True)
    
test_df

,Postalcode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [13]:
toronto_df_grouped.shape

(103, 3)

In [14]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [15]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [16]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
geo_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geo_df, toronto_df_grouped, on='Postalcode')

In [18]:
geo_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [19]:
geo_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
toronto_data=geo_data[geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [21]:
CLIENT_ID = 'U4TBO1XIEEJXXMXNNE1PPNFIEI3TZMQ0CLI3ZIZVY0FSB5XH' # your Foursquare ID
CLIENT_SECRET = 'UKYDO1DUHZKNMLVMXULS2NG2DWHWSYJTZQDPX0BEGD3JDAHM' # your Foursquare Secret
VERSION = '20180604'

In [22]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=2000
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'NeighborhoodLatitude', 
                  'NeighborhoodLongitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [24]:
toronto_venues.head()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
3,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
4,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop


In [25]:
toronto_venues["Neighborhood"].groupby(toronto_venues["Neighborhood"]).agg('count')

Neighborhood
Berczy Park                                                                                                   100
Brockton, Parkdale Village, Exhibition Place                                                                  100
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                          100
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport    100
Central Bay Street                                                                                            100
Christie                                                                                                      100
Church and Wellesley                                                                                          100
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['VenueCategory'].unique())))

There are 231 uniques categories.


In [27]:
# print out the list of categories
toronto_venues['VenueCategory'].unique()

array(['Trail', 'Gastropub', 'Vegetarian / Vegan Restaurant',
       'Indie Movie Theater', 'Ice Cream Shop', 'Bakery', 'Bagel Shop',
       'Park', 'Toy / Game Store', 'Health Food Store', 'Coffee Shop',
       'French Restaurant', 'Beach', 'BBQ Joint', 'Pub', 'Breakfast Spot',
       'Nail Salon', 'Mexican Restaurant', 'Skating Rink',
       'Middle Eastern Restaurant', 'Juice Bar', 'Japanese Restaurant',
       'Liquor Store', 'Cupcake Shop', 'Burger Joint',
       'Mediterranean Restaurant', 'Tea Room', 'Diner', 'Café',
       'Hungarian Restaurant', 'Bar', 'Caribbean Restaurant', 'Gym',
       'Dog Run', 'Pharmacy', 'Grocery Store', 'Indian Restaurant',
       'Golf Course', 'Playground', 'Thai Restaurant',
       'Fish & Chips Shop', 'Sandwich Place', 'Ramen Restaurant',
       'Greek Restaurant', 'Chocolate Shop', 'Gaming Cafe', 'Bank',
       'Bookstore', 'Sushi Restaurant', 'Pizza Place', 'Asian Restaurant',
       'Fried Chicken Joint', 'Beer Store', 'Pool', 'Butcher',
      

In [28]:
Restaurant=toronto_venues[toronto_venues.VenueCategory=="Restaurant"]

In [29]:
Restaurant

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
107,"The Danforth West, Riverdale",43.679557,-79.352188,Rikkochez,43.677267,-79.353274,Restaurant
195,"The Danforth West, Riverdale",43.679557,-79.352188,Murgatroid,43.667381,-79.369311,Restaurant
324,Studio District,43.659526,-79.340923,Ruby WatchCo.,43.659149,-79.349170,Restaurant
361,Studio District,43.659526,-79.340923,Impact Kitchen,43.656369,-79.356980,Restaurant
442,Lawrence Park,43.728020,-79.388790,Sushi Shop,43.713861,-79.400093,Restaurant
472,Lawrence Park,43.728020,-79.388790,Trapper's Restaurant,43.734405,-79.404458,Restaurant
531,Davisville North,43.712751,-79.390197,The Keg Steakhouse + Bar - Yonge + Eglinton,43.705702,-79.397905,Restaurant
541,Davisville North,43.712751,-79.390197,Noorden Food x Bar,43.702814,-79.397645,Restaurant
560,Davisville North,43.712751,-79.390197,L'Avenue Bistro,43.705462,-79.375131,Restaurant
633,"North Toronto West, Lawrence Park",43.715383,-79.405678,The Keg Steakhouse + Bar - Yonge + Eglinton,43.705702,-79.397905,Restaurant


### ANALYZE EACH NEIGHBORHOOD

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(3883, 232)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 232)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,0.00,0.010000,0.00,0.00,0.01,0.02,0.00,0.000000,0.00,...,0.00,0.00,0.00,0.01,0.000000,0.00,0.000000,0.00,0.000000,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.020000,0.00,0.00,0.00,0.01,0.01,0.020000,0.03,...,0.00,0.00,0.00,0.01,0.010000,0.00,0.010000,0.00,0.000000,0.00
2,"Business reply mail Processing Centre, South C...",0.00,0.030000,0.00,0.01,0.00,0.00,0.01,0.010000,0.00,...,0.00,0.00,0.00,0.00,0.010000,0.00,0.000000,0.00,0.000000,0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.01,0.010000,0.00,0.00,0.01,0.01,0.00,0.000000,0.01,...,0.01,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.020000,0.00
4,Central Bay Street,0.00,0.010000,0.00,0.00,0.00,0.01,0.03,0.000000,0.00,...,0.00,0.00,0.00,0.01,0.010000,0.00,0.000000,0.00,0.030000,0.00
5,Christie,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.010000,0.00,...,0.00,0.00,0.00,0.04,0.000000,0.01,0.010000,0.00,0.000000,0.00
6,Church and Wellesley,0.00,0.010000,0.00,0.00,0.00,0.00,0.01,0.000000,0.00,...,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.010000,0.00
7,"Commerce Court, Victoria Hotel",0.00,0.010000,0.00,0.00,0.01,0.02,0.01,0.000000,0.00,...,0.00,0.00,0.00,0.01,0.000000,0.00,0.000000,0.00,0.010000,0.00
8,Davisville,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.010000,0.00,...,0.01,0.00,0.00,0.01,0.010000,0.00,0.010000,0.01,0.030000,0.00
9,Davisville North,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.010000,0.00,...,0.00,0.00,0.00,0.01,0.010000,0.00,0.010000,0.01,0.020000,0.00


### Create a new DataFrame for Restaurant

In [32]:
restaurant_toronto = toronto_grouped[["Neighborhoods","Restaurant"]]
restaurant_toronto

,Neighborhoods,Restaurant
0,Berczy Park,0.040000
1,"Brockton, Parkdale Village, Exhibition Place",0.040000
2,"Business reply mail Processing Centre, South C...",0.010000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.010000
4,Central Bay Street,0.050000
5,Christie,0.030000
6,Church and Wellesley,0.040000
7,"Commerce Court, Victoria Hotel",0.040000
8,Davisville,0.040000
9,Davisville North,0.030000


### Cluster Neighborhoods

In [33]:
# set number of clusters
kclusters = 3

toronto_clustering = restaurant_toronto.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 2, 0, 1, 0, 0, 0, 1], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = restaurant_toronto.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

In [35]:
toronto_merged

,Neighborhoods,Restaurant,Cluster Labels
0,Berczy Park,0.040000,0
1,"Brockton, Parkdale Village, Exhibition Place",0.040000,0
2,"Business reply mail Processing Centre, South C...",0.010000,2
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.010000,2
4,Central Bay Street,0.050000,0
5,Christie,0.030000,1
6,Church and Wellesley,0.040000,0
7,"Commerce Court, Victoria Hotel",0.040000,0
8,Davisville,0.040000,0
9,Davisville North,0.030000,1


In [36]:
toronto_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_data.set_index("Neighborhood"), on="Neighborhood")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 7)


,Neighborhood,Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,Berczy Park,0.04,0,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.04,0,M6K,West Toronto,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",0.01,2,M7Y,East Toronto,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.01,2,M5V,Downtown Toronto,43.628947,-79.394420
4,Central Bay Street,0.05,0,M5G,Downtown Toronto,43.657952,-79.387383


In [37]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 7)


,Neighborhood,Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,Berczy Park,0.040000,0,M5E,Downtown Toronto,43.644771,-79.373306
22,"Parkdale, Roncesvalles",0.040000,0,M6R,West Toronto,43.648960,-79.456325
24,"Regent Park, Harbourfront",0.060000,0,M5A,Downtown Toronto,43.654260,-79.360636
27,Roselawn,0.040000,0,M5N,Central Toronto,43.711695,-79.416936
29,St. James Town,0.040000,0,M5C,Downtown Toronto,43.651494,-79.375418
37,"Toronto Dominion Centre, Design Exchange",0.050000,0,M5K,Downtown Toronto,43.647177,-79.381576
31,Stn A PO Boxes,0.040000,0,M5W,Downtown Toronto,43.646435,-79.374846
8,Davisville,0.040000,0,M4S,Central Toronto,43.704324,-79.388790
30,"St. James Town, Cabbagetown",0.040000,0,M4X,Downtown Toronto,43.667967,-79.367675
6,Church and Wellesley,0.040000,0,M4Y,Downtown Toronto,43.665860,-79.383160


### Visualization the cluster

In [38]:
import folium

In [39]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.6534817, -79.3839347.


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## EXAMINE RESULT

In [41]:
# CLUSTER 0
toronto_merged[toronto_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
0,Berczy Park,0.04,0,M5E,Downtown Toronto,43.644771,-79.373306
22,"Parkdale, Roncesvalles",0.04,0,M6R,West Toronto,43.648960,-79.456325
24,"Regent Park, Harbourfront",0.06,0,M5A,Downtown Toronto,43.654260,-79.360636
27,Roselawn,0.04,0,M5N,Central Toronto,43.711695,-79.416936
29,St. James Town,0.04,0,M5C,Downtown Toronto,43.651494,-79.375418
37,"Toronto Dominion Centre, Design Exchange",0.05,0,M5K,Downtown Toronto,43.647177,-79.381576
31,Stn A PO Boxes,0.04,0,M5W,Downtown Toronto,43.646435,-79.374846
8,Davisville,0.04,0,M4S,Central Toronto,43.704324,-79.388790
30,"St. James Town, Cabbagetown",0.04,0,M4X,Downtown Toronto,43.667967,-79.367675
6,Church and Wellesley,0.04,0,M4Y,Downtown Toronto,43.665860,-79.383160


In [42]:
# CLUSTER 1
toronto_merged[toronto_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
32,Studio District,0.020000,1,M4M,East Toronto,43.659526,-79.340923
28,"Runnymede, Swansea",0.020000,1,M6S,West Toronto,43.651571,-79.484450
34,"The Annex, North Midtown, Yorkville",0.030000,1,M5R,Central Toronto,43.672710,-79.405678
26,Rosedale,0.030000,1,M4W,Downtown Toronto,43.679563,-79.377529
25,"Richmond, Adelaide, King",0.030000,1,M5H,Downtown Toronto,43.650571,-79.384568
36,"The Danforth West, Riverdale",0.020000,1,M4K,East Toronto,43.679557,-79.352188
23,"Queen's Park, Ontario Provincial Government",0.030000,1,M7A,Downtown Toronto,43.662301,-79.389494
21,"North Toronto West, Lawrence Park",0.020000,1,M4R,Central Toronto,43.715383,-79.405678
19,"Little Portugal, Trinity",0.030000,1,M6J,West Toronto,43.647927,-79.419750
18,Lawrence Park,0.022727,1,M4N,Central Toronto,43.728020,-79.388790


In [43]:
# CLUSTER 2
toronto_merged[toronto_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Postalcode,Borough,Latitude,Longitude
2,"Business reply mail Processing Centre, South C...",0.010000,2,M7Y,East Toronto,43.662744,-79.321558
35,The Beaches,0.000000,2,M4E,East Toronto,43.676357,-79.293031
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.010000,2,M5V,Downtown Toronto,43.628947,-79.394420
10,"Dufferin, Dovercourt Village",0.010526,2,M6H,West Toronto,43.669005,-79.442259
14,"Harbourfront East, Union Station, Toronto Islands",0.010000,2,M5J,Downtown Toronto,43.640816,-79.381752
16,"India Bazaar, The Beaches West",0.000000,2,M4L,East Toronto,43.668999,-79.315572
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.010000,2,M4V,Central Toronto,43.686412,-79.400049
38,"University of Toronto, Harbord",0.010000,2,M5S,Downtown Toronto,43.662696,-79.400049
